## Efficient Yelp API Calls (Core)
- Brian Lafferty
- 7.26.2022

### Loading Libraries

In [9]:
# Imports and libraries
import numpy as np
import pandas as pd
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### loading and saving credentials for Yelp

In [10]:
# loading credentials from secret folder
with open('/Users/laffe/.secret/yelp_api.json') as f:
    login = json.load(f)
# instantiate YelpAPI variable and give it my key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### define search area and terms

In [11]:
# setting the variables to the parameters of the API
LOCATION = 'Whispering Pines, NC, 28327'
TERM = 'Pizza'

### creating a file and folder to store search

In [12]:
# creating a folder to save the data in
folder_name = "Data/"
os.makedirs(folder_name, exist_ok = True)

In [13]:
# using the new folder to create file path, with the search terms as the name
JSON_FILE = folder_name+f"{LOCATION.split(',')[0]}-{TERM}.json"

In [14]:
# check to see if JSON_FILE exists already
file_exists = os.path.isfile(JSON_FILE)
# if it does oot exist
if file_exists == False:
    # create any needed folders
    # get the folder name only
    folder = os.path.dirname(JSON_FILE)
    # if JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
    # tell users and save empty list
    print(f"This {JSON_FILE} was not found! Saving empty list to file.")
    
    # save the empty file
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
# if it exits, inform user
else:
    print(f"This {JSON_FILE} already exists.")

This Data/Whispering Pines-Pizza.json already exists.


In [16]:
# loading the previous file and finding the length, this will be used for the offset
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

# set offset based on previous results
num_results = len(previous_results)
print(f'- {num_results} previous results found.')

- 0 previous results found.


In [17]:
# using yelp_api variables and search_query method to perform the API call
results = yelp_api.search_query(location = LOCATION, term = TERM, offset = num_results)


In [18]:
# how many total results were found
total_results = results['total']
print(f'The total number of results from the search was {total_results}')
# how many businesses were returned
results_per_page = len(results['businesses'])


The total number of results from the search was 32


In [19]:
# use math.ceil to round up for the total number of pages of results
num_pages = math.ceil((results['total']-num_results)/ results_per_page)
num_pages

2

In [20]:
# adding the previous search to the save file using extend
previous_results.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(previous_results, f)

In [21]:
# creating a for loop to run the needed api calls to get the full list
# also using the tqdm notebook to practice making a status bar
for i in tqdm_notebook(range(1, num_pages + 1)):
    time.sleep(.2)
    # read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    # save the number of results for use as the offset
    num_results = len(previous_results)
    # using number of results as the offset
    results = yelp_api.search_query(location = LOCATION, term = TERM, offset = num_results)
    
    # append new results and save eh file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)

  0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# converting the results from .json to a dataframe
pizza_df = pd.read_json(JSON_FILE)
pizza_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,dyRxi0pGj9A4HXLodN3kDg,whispering-pies-carthage,Whispering Pies,https://s3-media4.fl.yelpcdn.com/bphoto/N2t7BX...,False,https://www.yelp.com/biz/whispering-pies-carth...,52,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.2285534977859, 'longitude': -7...",[delivery],$$,"{'address1': '334 Mill Creek Rd', 'address2': ...",+19107252255,(910) 725-2255,3094.257397
1,mTzEyO8SlpWjEeFsHGUBzA,pinehurst-pizza-pinehurst,Pinehurst Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/B9xBHQ...,False,https://www.yelp.com/biz/pinehurst-pizza-pineh...,72,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.2007963174076, 'longitude': -7...",[delivery],$$,"{'address1': '15 Mc Intyre Rd', 'address2': ''...",+19102550088,(910) 255-0088,9864.807167
2,9LEunCMhTr6l0ksl-2_7DA,filly-and-colts-whispering-pines,Filly & Colts,https://s3-media3.fl.yelpcdn.com/bphoto/zkpnqR...,False,https://www.yelp.com/biz/filly-and-colts-whisp...,16,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 35.23833, 'longitude': -79.37308}","[pickup, delivery]",$$,"{'address1': '26 Sandpiper Dr', 'address2': No...",+19106887125,(910) 688-7125,1786.153990
3,63Ry46VCS7TEFI4sRWMA-Q,the-block-southern-pines,The Block,https://s3-media3.fl.yelpcdn.com/bphoto/KSi0yL...,False,https://www.yelp.com/biz/the-block-southern-pi...,22,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 35.17571, 'longitude': -79.39411}",[],NaN,"{'address1': '290 W Pennsylvania Ave', 'addres...",+19107257024,(910) 725-7024,8643.137406
4,p9wieO9ZNd4GgGn8hiwEbA,valentis-vass-2,Valenti's,https://s3-media2.fl.yelpcdn.com/bphoto/bscbNU...,False,https://www.yelp.com/biz/valentis-vass-2?adjus...,77,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 35.24920324906586, 'longitude': -...",[delivery],$$,"{'address1': '3781 US Hwy 1', 'address2': '', ...",+19102451105,(910) 245-1105,8194.233721
